In [9]:
import pickle

import numpy as np
from tqdm import tqdm

In [10]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [11]:
with open(f'ptv_result_test.pickle', mode='rb') as fp:
    ptv_result = pickle.load(fp)

In [12]:
data = [[[]*50]*50]*50

In [13]:
for p in tqdm(range(ptv_result.shape[0])):
    for j in range(0, 50):
        for i in range(0, 50):
            if np.sqrt((1350 // 50 * i - ptv_result.iloc[p, 0])**2 + (1300 // 50 * j - ptv_result.iloc[p, 1])**2) < 75:
                data[j][i].append(
                    [
                        ptv_result.iloc[p, 0],
                        ptv_result.iloc[p, 1],
                        ptv_result.iloc[p, 2],
                        ptv_result.iloc[p, 3]
                    ]
                )
                break
        else:
            continue
        break

0it [00:00, ?it/s]


In [14]:
# with open(f'ptv_result2.pickle', mode='rb') as fp:
#     data = pickle.load(fp)

In [15]:
u = np.zeros([1300 // 50, 1300 // 50])
v = np.zeros([1300 // 50, 1300 // 50])

In [16]:
for j in range(50):
    for i in range(50):
        tmp = data[j][i]
        top_u, top_v = 0, 0
        bottom = 0

        sigma = 1.24 * np.sqrt(75*75*np.pi / 1392032)

        for k in range(len(tmp)):
            r = np.sqrt((1350 // 50 * i - tmp[k][0])**2 + (1300 // 50 * j - tmp[k][1])**2)
            omega = np.exp(- (r / sigma)**2)

            top_u += omega * tmp[k][2]
            top_v += omega * tmp[k][3]
            bottom += omega

        u[j, i] = top_u / bottom
        v[j, i] = top_v / bottom

ZeroDivisionError: division by zero